In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import clear_output
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.applications import EfficientNetB0
import tensorflow as tf

In [3]:
label = pd.read_csv('/content/drive/MyDrive/Mango/cp_label.csv')

In [4]:
clas = ['Bad_color', 'Anthrax', 'Milk_A', 'Machine_harm', 'Black_spot']

In [5]:
conditions = label['Condition']
new_labeled = [clas.index(con) for con in conditions]

In [6]:
label['Condition'] = new_labeled

In [8]:
path = '/content/drive/MyDrive/Mango/cp_train/'
iter = [i for i in range(len(label))]
X = []
Y = []

cnt = 0
for i in random.sample(iter, len(iter)):
    img = cv2.imread(path+label.loc[i, 'Img'])
    img = cv2.resize(img, (300, 300), interpolation=cv2.INTER_LINEAR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #img = tf.io.gfile.GFile(path+label.loc[i, 'Img'], 'rb').read()
    #img = tf.image.decode_jpeg(img)
    #img = tf.image.resize(img, [300, 300], method=tf.image.ResizeMethod.BILINEAR)
    #res = keras.preprocessing.image.img_to_array(img)
    X.append(img)
    Y.append(label.loc[i, 'Condition'])

    cnt += 1
    clear_output(wait=True)
    perc = (cnt/len(iter))*100
    print('%.2f'%perc, '%')
    print(i, cnt)

0.09 %
4873 10


KeyboardInterrupt: ignored

In [ ]:
print(len(X), len(Y))

10882 10882


In [ ]:
X = np.array(X)
Y = np.array(Y)
for i in range(len(X)):
    X[i] = X[i].astype(float)

In [ ]:
Y = keras.utils.to_categorical(Y, num_classes=5)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
print(len(x_train), len(x_test))

8705 2177


MODEL

In [ ]:
type(x_train[0])

tensorflow.python.framework.ops.EagerTensor

In [ ]:
from tensorflow.keras.applications import EfficientNetB3

# build model
inputs = layers.Input(shape=(300, 300, 3))
outputs = EfficientNetB3(include_top=True, weights=None, classes=5)(inputs)

model = tf.keras.Model(inputs, outputs)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300, 300, 3)]     0         
_________________________________________________________________
efficientnetb3 (Functional)  (None, 5)                 10791220  
Total params: 10,791,220
Trainable params: 10,703,917
Non-trainable params: 87,303
_________________________________________________________________


In [ ]:
# train model
batch_size = 16
epochs = 30
CB = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, mode='min')
hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split = 0.2, callbacks = [CB], verbose=1)

ValueError: ignored